# BiLSTM

In [1]:
from reader import Reader
from wordembedder import Wordembedder
import nltk
import pandas as pd
import numpy as np
from sentence_splitter import SentenceSplitter
from keras import regularizers
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Activation, Embedding
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras_contrib.layers import CRF
from keras_contrib import metrics as mt
from keras_contrib import losses as ls

Using TensorFlow backend.


In [2]:
datas = pd.read_csv('data-extraction.csv')
datas.head()

,id,texts,tags
0,0,Nuclear theory devoted major efforts since 4 d...,"['B-PHRASE', 'I-PHRASE', 'OTHER', 'B-PHRASE', ..."
1,1,The next important step might be the derivatio...,"['B-PHRASE', 'OTHER', 'B-PHRASE', 'OTHER', 'B-..."
2,2,A fluctuating vacuum is a general feature of q...,"['B-PHRASE', 'I-PHRASE', 'OTHER', 'B-PHRASE', ..."
3,3,We have presented spectrally resolved femtosec...,"['B-PHRASE', 'OTHER', 'B-PHRASE', 'I-PHRASE', ..."
4,4,"Water is the most important liquid, and the na...","['B-PHRASE', 'OTHER', 'B-PHRASE', 'OTHER', 'B-..."


In [3]:
# reader = Reader()
# reader.read(r"C:\Users\user\Documents\IF5282-NLP\relation-extraction\data\scienceie2017_train\train2")

In [4]:
# reader.publications[0].text

In [5]:
X = datas['texts']
y = datas['tags']
docs = []
for pub in X:
    docs.append(pub)
wv_doc = Wordembedder(docs, filename='embedding_matrix.npy')
encoded_docs = wv_doc.encoded_docs
vocab_size=wv_doc.vocab_size

In [6]:
y_doc = wv_doc.encodeDocs(y)

In [7]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
y_vec = np_utils.to_categorical(encoded_Y)
y_doc = y_doc.reshape(y_doc.shape[0], y_doc.shape[1], 1)
print(y_doc.shape)

X_train, X_test, y_train, y_test = train_test_split(encoded_docs, y_doc, test_size=0.2, random_state=1)
print(len(X_train), len(y_train))

(350, 271, 1)
280 280


In [8]:
def fit_and_eval_bilstm(X_train, y_train, X_test, y_test, X_val, y_val, vocab_size, embedding_matrix, embedding_dim, max_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 300, weights=[wv_doc.embedding_matrix], input_length=wv_doc.max_length))
    model.add(Bidirectional(LSTM(units=256, recurrent_dropout=0.3, return_sequences=True, input_shape=(1, wv_doc.max_length, 1))))
    #model.add(Bidirectional(LSTM(units=128, recurrent_dropout=0.2, dropout=0.2)))
    n_tags = 271
    crf = CRF(n_tags, sparse_target=True)
    model.add(crf)
    #model.add(Dense(350, activation='softmax'))
    model.compile(optimizer='rmsprop', loss=ls.crf_loss, metrics=[mt.crf_accuracy])
    result = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_val, y_val), verbose=1)
    metrics = model.evaluate(x_test, y_test)
    return result, metrics[0], metrics[1]

In [124]:
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[wv_doc.embedding_matrix], input_length=wv_doc.max_length))
model.add(Bidirectional(LSTM(units=256, recurrent_dropout=0.3, return_sequences=True, input_shape=(1, wv_doc.max_length, 1))))
model.add(Bidirectional(LSTM(units=128, recurrent_dropout=0.2, dropout=0.2)))
model.add(Dense(350, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_39 (Embedding)     (None, 271, 300)          2557200   
_________________________________________________________________
bidirectional_50 (Bidirectio (None, 271, 512)          1140736   
_________________________________________________________________
bidirectional_51 (Bidirectio (None, 256)               656384    
_________________________________________________________________
dense_16 (Dense)             (None, 350)               89950     
Total params: 4,444,270
Trainable params: 4,444,270
Non-trainable params: 0
_________________________________________________________________


In [ ]:
kfold = KFold(5, True, random_state=1)
model_history = []
metrics = []
i=1
for train, val in kfold.split(X_train, y_train):
    print("Training on Fold: ",i)
    train_x = X_train[train]
    val_x = X_train[val]
    train_y = y_train[train]
    val_y = y_train[val]
    
    history, loss, accuracy = fit_and_eval_bilstm(train_x, train_y, X_test, y_test, val_x, val_y, vocab_size, wv_doc.embedding_matrix, 300, 271)
    model_history.append(history)
    metrics.append([loss, accuracy])
    i+=1

Training on Fold:  1
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 224 samples, validate on 56 samples
Epoch 1/10
